In [ ]:
import math, re, os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from kaggle_datasets import KaggleDatasets
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.applications import EfficientNetB7, EfficientNetB5
from keras.applications.densenet import DenseNet201
from sklearn import metrics
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
# Data access
GCS_DS_PATH = KaggleDatasets().get_gcs_path("plant-pathology-2021-fgvc8")

# Configuration
EPOCHS = 20
BATCH_SIZE = 8 * strategy.num_replicas_in_sync
IM_Z = 768

In [ ]:
def format_train_path(fname):
    return GCS_DS_PATH+'/train_images/'+fname

def format_test_path(fname):
    return GCS_DS_PATH+'/test_images/'+fname

In [ ]:
train_dir="../input/plant-pathology-2021-fgvc8/train_images/"
test_dir="../input/plant-pathology-2021-fgvc8/test_images/"
df_train=pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
df_sub = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')

In [ ]:
train_paths = df_train.image.apply(format_train_path)
test_paths = df_sub.image.apply(format_test_path)

In [ ]:
labels = df_train['labels'].str.split(" ").apply(pd.Series, 1).stack()
labels.index = labels.index.droplevel(-1)
target_labels = pd.get_dummies(labels).groupby(level=0).sum()
target_labels.head()

In [ ]:
train_paths, valid_paths, train_labels, valid_labels = train_test_split(
    train_paths, target_labels, test_size=0.2)

In [ ]:
def decode_image(filename, label=None, image_size=(IM_Z, IM_Z)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)
    
    if label is None:
        return image
    else:
        return image, label

def data_augment(image, label=None):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
#     image = tf.image.adjust_brightness(image, delta=0.2)
#     image = tf.image.adjust_contrast(image,2)
    
    if label is None:
        return image
    else:
        return image, label

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_paths, train_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .cache()
    .map(data_augment, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(512)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((valid_paths, valid_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(test_paths)
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
)

In [ ]:
def build_lrfn(lr_start=0.00001, lr_max=0.000075, 
               lr_min=0.000001, lr_rampup_epochs=20, 
               lr_sustain_epochs=0, lr_exp_decay=.8):
    lr_max = lr_max * strategy.num_replicas_in_sync

    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) * lr_exp_decay**(epoch - lr_rampup_epochs - lr_sustain_epochs) + lr_min
        return lr
    
    return lrfn

ch_p = ModelCheckpoint(filepath="model_ef.h5", monitor='val_loss', save_weights_only=True,
                                                 verbose=1)

In [ ]:
with strategy.scope():
    model = tf.keras.Sequential([
        EfficientNetB7(
            input_shape=(IM_Z, IM_Z, 3),
            weights=None,
            include_top=False
        ),
        L.GlobalAveragePooling2D(),
        L.Dense(train_labels.shape[1], activation='softmax')
    ])
        
    model.compile(
        optimizer='adam',
        loss = 'categorical_crossentropy',
        metrics=['categorical_accuracy']
    )
#     model.summary()

model.load_weights('../input/plant-2021-densenet201-efficientnetb7-tpu/model_ef.h5')

In [ ]:
lrfn = build_lrfn()
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)
STEPS_PER_EPOCH = train_labels.shape[0] // BATCH_SIZE 

In [ ]:
history = model.fit(
    train_dataset, 
    epochs=EPOCHS, 
    callbacks=[lr_schedule, ch_p],
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=valid_dataset
)

In [ ]:
with strategy.scope():
    model1 = tf.keras.Sequential([
        tf.keras.applications.DenseNet201(
            input_shape=(IM_Z, IM_Z, 3),
            weights=None,
            include_top=False
        ),
        L.GlobalAveragePooling2D(),
        L.Dense(train_labels.shape[1], activation='softmax')
    ])
        
             
    model1.compile(
        optimizer='adam',
        loss = 'categorical_crossentropy',
        metrics=['categorical_accuracy']
    )
#     model.summary()

model1.load_weights('../input/plant-2021-densenet201-efficientnetb7-tpu/model_den.h5')

In [ ]:
ch_p_den = ModelCheckpoint(filepath="model_den.h5", monitor='val_loss', save_weights_only=True,
                                                 verbose=1)

In [ ]:
history1 = model1.fit(
    train_dataset, 
    epochs=EPOCHS, 
    callbacks=[lr_schedule, ch_p_den],
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=valid_dataset
)

In [ ]:
probs = (model1.predict(test_dataset)+model.predict(test_dataset))/2

In [ ]:
label_names = target_labels.columns
def get_labels_from_pred(preds):
    lab = []
    for row in preds:
        l = []
        for i,v in enumerate(row):
            if v >= 0.4:
                l.append(i)
        lab.append(" ".join(label_names[l]))
    return lab

In [ ]:
pred_labels = get_labels_from_pred(probs)

In [ ]:
df_sub["labels"] = pred_labels

In [ ]:
df_sub.head()

In [ ]:
df_sub.to_csv("submission1.csv", index=False)